# ETL de datos de importación de productos

## Instalación de librerías base

In [1]:
import pandas as pd
from sqlalchemy import create_engine
import uuid
import psycopg2 as pg

## Extraction

In [2]:
#The dataset cotain information retaled to importation and exportation transactions between countries.
# Conexión a la base de datos PostgreSQL

conn = pg.connect(
    host="localhost",
    database="postgres",
    user="postgres",
    password="mysecretpass"
)
query = "SELECT * FROM trades"
trades_generator = pd.read_sql_query(query, conn)

trades_generator.head(5)

,country_code,year,comm_code,flow,trade_usd,kg,quantity,quantity_name
0,SYC,1998,890200,Import,1431426.0,0.0,23000.0,Number of items
1,SYC,1998,890310,Import,31406.0,0.0,2545.0,Number of items
2,SYC,1998,890310,Export,950.0,0.0,300.0,Number of items
3,SYC,1998,890310,Re-Export,950.0,0.0,300.0,Number of items
4,SYC,1998,890391,Import,18251.0,0.0,450.0,Number of items


In [3]:
#Informatin related with the codes that represent each coutnry
df_countries = pd.read_json('C:/Users/manue/Desktop/Platzi/Fundamentos de ETL con Python y Pentaho/country_data.json')

#Codes that are used in the importation and exportation transactions.
df_codes = pd.read_csv('C:/Users/manue/Desktop/Platzi/Fundamentos de ETL con Python y Pentaho/hs_codes.csv')
df_codes.head(20)

,Order,Level,Code,Parent,Code_comm,Parent.1,Description_complex,Description
0,1654555,1,0,NaN,NaN,NaN,This classification has been uploaded in RAMON...,This classification has been uploaded in RAMON...
1,1654556,1,10011000090,NaN,I,NaN,SECTION I - LIVE ANIMALS; ANIMAL PRODUCTS,LIVE ANIMALS; ANIMAL PRODUCTS
2,1654557,2,10021000090,1.001100e+10,1,I,CHAPTER 1 - LIVE ANIMALS,LIVE ANIMALS
3,1654558,3,10100000080,1.002100e+10,1.01,1,"Live horses, asses, mules and hinnies","Live horses, asses, mules and hinnies"
4,1654559,4,10121000010,1.010000e+10,NaN,1.01,- Horses,NaN
5,1654560,5,10121000080,1.012100e+10,101.21,NaN,-- Pure-bred breeding animals,Pure-bred breeding horses
6,1654561,5,10129000080,1.012100e+10,101.29,NaN,-- Other,Live horses (excl. pure-bred for breeding)
7,1654562,4,10130000080,1.010000e+10,101.3,1.01,- Asses,Live asses
8,1654563,4,10190000080,1.010000e+10,101.9,1.01,- Other,Live mules and hinnies
9,1654564,3,10200000080,1.002100e+10,1.02,1,Live bovine animals,Live bovine animals


In [4]:
df_parents = df_codes[df_codes['Level']==2].copy()
df_parents.head(20)

,Order,Level,Code,Parent,Code_comm,Parent.1,Description_complex,Description
2,1654557,2,10021000090,1.001100e+10,1,I,CHAPTER 1 - LIVE ANIMALS,LIVE ANIMALS
52,1654607,2,20021000090,1.001100e+10,2,I,CHAPTER 2 - MEAT AND EDIBLE MEAT OFFAL,MEAT AND EDIBLE MEAT OFFAL
140,1654695,2,30021000090,1.001100e+10,3,I,"CHAPTER 3 - FISH AND CRUSTACEANS, MOLLUSCS AND...","FISH AND CRUSTACEANS, MOLLUSCS AND OTHER AQUAT..."
416,1654971,2,40021000090,1.001100e+10,4,I,CHAPTER 4 - DAIRY PRODUCE; BIRDS' EGGS; NATURA...,DAIRY PRODUCE; BIRDS' EGGS; NATURAL HONEY; EDI...
463,1655018,2,50021000090,1.001100e+10,5,I,"CHAPTER 5 - PRODUCTS OF ANIMAL ORIGIN, NOT ELS...","PRODUCTS OF ANIMAL ORIGIN, NOT ELSEWHERE SPECI..."
486,1655041,2,60021000090,6.001100e+10,6,II,CHAPTER 6 - LIVE TREES AND OTHER PLANTS; BULBS...,"LIVE TREES AND OTHER PLANTS; BULBS, ROOTS AND ..."
508,1655063,2,70021000090,6.001100e+10,7,II,CHAPTER 7 - EDIBLE VEGETABLES AND CERTAIN ROOT...,EDIBLE VEGETABLES AND CERTAIN ROOTS AND TUBERS
596,1655151,2,80021000090,6.001100e+10,8,II,CHAPTER 8 - EDIBLE FRUIT AND NUTS; PEEL OF CIT...,EDIBLE FRUIT AND NUTS; PEEL OF CITRUS FRUIT OR...
690,1655245,2,90021000090,6.001100e+10,9,II,"CHAPTER 9 - COFFEE, TEA, MATÉ AND SPICES","COFFEE, TEA, MATÉ AND SPICES"
752,1655307,2,100021000090,6.001100e+10,10,II,CHAPTER 10 - CEREALS,CEREALS


## Cleanning codes data set

In [5]:
df_codes = df_codes[df_codes['Code_comm'].notnull()]

In [7]:
def clean_code(text):
    text = str(text)
    parent_code = None
    if len(text) == 11:
        code = text[:5]
        parent_code = text[:1]
    else:
        code = text[:6]
        parent_code = text[:2]
    try:
        parent = df_parents[df_parents['Code_comm']==parent_code]['Description'].values[0]
        
    except:
        parent = None
        
    return (code,parent)

In [8]:
df_codes[['clean_code','parent_description']] = df_codes.apply(lambda x: clean_code(x['Code']), axis=1, result_type='expand')
df_codes.head()

,Order,Level,Code,Parent,Code_comm,Parent.1,Description_complex,Description,clean_code,parent_description
1,1654556,1,10011000090,NaN,I,NaN,SECTION I - LIVE ANIMALS; ANIMAL PRODUCTS,LIVE ANIMALS; ANIMAL PRODUCTS,10011,LIVE ANIMALS
2,1654557,2,10021000090,1.001100e+10,1,I,CHAPTER 1 - LIVE ANIMALS,LIVE ANIMALS,10021,LIVE ANIMALS
3,1654558,3,10100000080,1.002100e+10,1.01,1,"Live horses, asses, mules and hinnies","Live horses, asses, mules and hinnies",10100,LIVE ANIMALS
5,1654560,5,10121000080,1.012100e+10,101.21,NaN,-- Pure-bred breeding animals,Pure-bred breeding horses,10121,LIVE ANIMALS
6,1654561,5,10129000080,1.012100e+10,101.29,NaN,-- Other,Live horses (excl. pure-bred for breeding),10129,LIVE ANIMALS


In [9]:
df_codes= df_codes[df_codes['clean_code'].notnull()][['clean_code','Description','parent_description']]
df_codes

,clean_code,Description,parent_description
1,10011,LIVE ANIMALS; ANIMAL PRODUCTS,LIVE ANIMALS
2,10021,LIVE ANIMALS,LIVE ANIMALS
3,10100,"Live horses, asses, mules and hinnies",LIVE ANIMALS
5,10121,Pure-bred breeding horses,LIVE ANIMALS
6,10129,Live horses (excl. pure-bred for breeding),LIVE ANIMALS
...,...,...,...
7432,970200,"Original engravings, prints and lithographs","WORKS OF ART, COLLECTORS' PIECES AND ANTIQUES"
7433,970300,"Original sculptures and statuary, in any material","WORKS OF ART, COLLECTORS' PIECES AND ANTIQUES"
7434,970400,"Postage or revenue stamps, stamp-postmarks, fi...","WORKS OF ART, COLLECTORS' PIECES AND ANTIQUES"
7435,970500,Collections and collector's pieces of zoologic...,"WORKS OF ART, COLLECTORS' PIECES AND ANTIQUES"


In [10]:
df_codes['id_code'] = df_codes.index
df_codes['clean_code'] = df_codes['clean_code'].astype('int64')
df_codes

,clean_code,Description,parent_description,id_code
1,10011,LIVE ANIMALS; ANIMAL PRODUCTS,LIVE ANIMALS,1
2,10021,LIVE ANIMALS,LIVE ANIMALS,2
3,10100,"Live horses, asses, mules and hinnies",LIVE ANIMALS,3
5,10121,Pure-bred breeding horses,LIVE ANIMALS,5
6,10129,Live horses (excl. pure-bred for breeding),LIVE ANIMALS,6
...,...,...,...,...
7432,970200,"Original engravings, prints and lithographs","WORKS OF ART, COLLECTORS' PIECES AND ANTIQUES",7432
7433,970300,"Original sculptures and statuary, in any material","WORKS OF ART, COLLECTORS' PIECES AND ANTIQUES",7433
7434,970400,"Postage or revenue stamps, stamp-postmarks, fi...","WORKS OF ART, COLLECTORS' PIECES AND ANTIQUES",7434
7435,970500,Collections and collector's pieces of zoologic...,"WORKS OF ART, COLLECTORS' PIECES AND ANTIQUES",7435


#### Cleanning countries data set

In [11]:
df_countries

,country,images_file,image_url,alpha-2,alpha-3,country-code,iso_3166-2,region,sub-region,intermediate-region,region-code,sub-region-code,intermediate-region-code
0,Afghanistan,Flag_of_Afghanistan.svg,https://upload.wikimedia.org/wikipedia/commons...,AF,AFG,4.0,ISO 3166-2:AF,Asia,Southern Asia,None,142.0,34.0,NaN
1,Albania,Flag_of_Albania.svg,https://upload.wikimedia.org/wikipedia/commons...,AL,ALB,8.0,ISO 3166-2:AL,Europe,Southern Europe,None,150.0,39.0,NaN
2,Algeria,Flag_of_Algeria.svg,https://upload.wikimedia.org/wikipedia/commons...,DZ,DZA,12.0,ISO 3166-2:DZ,Africa,Northern Africa,None,2.0,15.0,NaN
3,Andorra,Flag_of_Andorra.svg,https://upload.wikimedia.org/wikipedia/commons...,AD,AND,20.0,ISO 3166-2:AD,Europe,Southern Europe,None,150.0,39.0,NaN
4,Angola,Flag_of_Angola.svg,https://upload.wikimedia.org/wikipedia/commons...,AO,AGO,24.0,ISO 3166-2:AO,Africa,Sub-Saharan Africa,Middle Africa,2.0,202.0,17.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
268,United States Minor Outlying Islands,None,None,UM,UMI,581.0,ISO 3166-2:UM,Oceania,Micronesia,None,9.0,57.0,NaN
269,Virgin Islands (British),None,None,VG,VGB,92.0,ISO 3166-2:VG,Americas,Latin America and the Caribbean,Caribbean,19.0,419.0,29.0
270,Virgin Islands (U.S.),None,None,VI,VIR,850.0,ISO 3166-2:VI,Americas,Latin America and the Caribbean,Caribbean,19.0,419.0,29.0
271,Wallis and Futuna,None,None,WF,WLF,876.0,ISO 3166-2:WF,Oceania,Polynesia,None,9.0,61.0,NaN


In [12]:
#df_countries = df_countries[['alpha-3','country','region','sub-region']]
#df_countries = df_countries[df_countries['alpha-3'].notnull()]
df_countries = df_countries[df_countries['alpha-3'].notnull()][['alpha-3','country','region','sub-region']]
df_countries['id_country'] = df_countries.index + 1
df_countries

,alpha-3,country,region,sub-region,id_country
0,AFG,Afghanistan,Asia,Southern Asia,1
1,ALB,Albania,Europe,Southern Europe,2
2,DZA,Algeria,Africa,Northern Africa,3
3,AND,Andorra,Europe,Southern Europe,4
4,AGO,Angola,Africa,Sub-Saharan Africa,5
...,...,...,...,...,...
268,UMI,United States Minor Outlying Islands,Oceania,Micronesia,269
269,VGB,Virgin Islands (British),Americas,Latin America and the Caribbean,270
270,VIR,Virgin Islands (U.S.),Americas,Latin America and the Caribbean,271
271,WLF,Wallis and Futuna,Oceania,Polynesia,272


### Merge

In [13]:
df_trades = trades_generator.copy()

In [14]:
df_trades_clean = df_trades.merge(df_codes[['clean_code','id_code']], how = 'left', left_on = 'comm_code', right_on = 'clean_code')

In [15]:
#df_trades_clean = df_trades_clean.merge(df_countries[['alpha-3','id_country']],how = 'left', left_on = 'country_code', right_on = 'alpha-3')
#I use the next way to merge, it allow the computer to manage in a better way the memory allocation.

df_trades_clean = pd.merge(df_trades_clean, df_countries[['alpha-3','id_country']], how='left', left_on='country_code', right_on='alpha-3')

In [16]:
df_trades_clean.head()

,country_code,year,comm_code,flow,trade_usd,kg,quantity,quantity_name,clean_code,id_code,alpha-3,id_country
0,SYC,1998,890200,Import,1431426.0,0.0,23000.0,Number of items,890200,6928,SYC,155
1,SYC,1998,890310,Import,31406.0,0.0,2545.0,Number of items,890310,6930,SYC,155
2,SYC,1998,890310,Export,950.0,0.0,300.0,Number of items,890310,6930,SYC,155
3,SYC,1998,890310,Re-Export,950.0,0.0,300.0,Number of items,890310,6930,SYC,155
4,SYC,1998,890391,Import,18251.0,0.0,450.0,Number of items,890391,6932,SYC,155


In [17]:
def  create_dimensions(data, id_name):
    list_keys = []
    value = 1
    for x in data:
        list_keys.append(value)
        value += 1       
    return pd.DataFrame({ id_name: list_keys ,'values' : data})
#df_trades_clean.head()

In [18]:
df_year = create_dimensions( df_trades_clean['year'].unique(),'id_year')
df_flow = create_dimensions( df_trades_clean['flow'].unique(),'id_flow')
df_quantity = create_dimensions( df_trades_clean['quantity_name'].unique(),'id_quantity_name')

df_trades_clean = df_trades_clean.merge(df_year, how = 'left', left_on = 'year', right_on= 'values')
df_trades_clean = df_trades_clean.merge(df_flow, how = 'left', left_on = 'flow', right_on= 'values')
df_trades_clean = df_trades_clean.merge(df_quantity, how = 'left', left_on = 'quantity_name', right_on= 'values')

In [19]:
df_trades_clean.head()

,country_code,year,comm_code,flow,trade_usd,kg,quantity,quantity_name,clean_code,id_code,alpha-3,id_country,id_year,values_x,id_flow,values_y,id_quantity_name,values
0,SYC,1998,890200,Import,1431426.0,0.0,23000.0,Number of items,890200,6928,SYC,155,1,1998,1,Import,1,Number of items
1,SYC,1998,890310,Import,31406.0,0.0,2545.0,Number of items,890310,6930,SYC,155,1,1998,1,Import,1,Number of items
2,SYC,1998,890310,Export,950.0,0.0,300.0,Number of items,890310,6930,SYC,155,1,1998,2,Export,1,Number of items
3,SYC,1998,890310,Re-Export,950.0,0.0,300.0,Number of items,890310,6930,SYC,155,1,1998,3,Re-Export,1,Number of items
4,SYC,1998,890391,Import,18251.0,0.0,450.0,Number of items,890391,6932,SYC,155,1,1998,1,Import,1,Number of items


In [20]:
df_trades_clean['id_trades'] = df_trades_clean.index  + 1
df_trades_final = df_trades_clean[['id_trades','trade_usd','kg','quantity','id_code','id_country','id_quantity_name','id_flow', 'id_year']].copy()

In [21]:
df_trades_final.head()

,id_trades,trade_usd,kg,quantity,id_code,id_country,id_quantity_name,id_flow,id_year
0,1,1431426.0,0.0,23000.0,6928,155,1,1,1
1,2,31406.0,0.0,2545.0,6930,155,1,1,1
2,3,950.0,0.0,300.0,6930,155,1,2,1
3,4,950.0,0.0,300.0,6930,155,1,3,1
4,5,18251.0,0.0,450.0,6932,155,1,1,1


In [22]:
df_countries = df_countries[['id_country','alpha-3','country','region','sub-region']]
df_codes = df_codes[['id_code','clean_code','Description']]

In [23]:
df_codes.head(10)

,id_code,clean_code,Description
1,1,10011,LIVE ANIMALS; ANIMAL PRODUCTS
2,2,10021,LIVE ANIMALS
3,3,10100,"Live horses, asses, mules and hinnies"
5,5,10121,Pure-bred breeding horses
6,6,10129,Live horses (excl. pure-bred for breeding)
7,7,10130,Live asses
8,8,10190,Live mules and hinnies
9,9,10200,Live bovine animals
11,11,10221,Pure-bred cattle for breeding
12,12,10229,Live cattle (excl. pure-bred for breeding)


### Clean trades

## Load

In [ ]:
import os
import boto3

